In [3]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
import time
import os
from datetime import datetime

# 현재 날짜를 yyyymmdd 형식으로 얻기
current_date = datetime.now().strftime("%Y%m%d")

# 사용자로부터 저장할 Excel 파일 기본 이름 입력받기
base_file_name = input("저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): ")

# 입력받은 파일 이름에 오늘 날짜 추가
excel_file_name = f"{base_file_name}_{current_date}.xlsx"

# 현재 작업 중인 폴더의 경로를 얻기
current_working_directory = os.getcwd()
file_path = os.path.join(current_working_directory, excel_file_name)  # 파일 경로 결합

# WebDriver 설정(Chrome 사용)
driver = webdriver.Chrome()

# Excel 설정
wb = Workbook()
ws = wb.create_sheet('검색어순위')
wb.remove(wb['Sheet'])  # 기본 생성된 시트 제거
ws.append(['순위', '인기검색어'])

# 네이버 데이터랩 쇼핑 인사이트 페이지로 이동
driver.get("https://datalab.naver.com/shoppingInsight/sCategory.naver")

# 필요한 요소가 로드될 때까지 대기하는 WebDriverWait 인스턴스 생성
wait = WebDriverWait(driver, 10)

# 드롭다운 및 옵션 선택 예시 (1번 옵션 선택)
dropdown1 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/span')))
dropdown1.click()
# 가구인테리어
option1 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/ul/li[5]')))
option1.click()
time.sleep(1)  # 드롭다운 선택 후 대기

# 추가 옵션 선택 (2번, 3번 등 필요한 경우에 따라 추가)
# 예: 2번 옵션 선택
dropdown2 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/span')))
dropdown2.click()
# 수예
option2 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/ul/li[16]/a')))
option2.click()
time.sleep(1)  # 드롭다운 선택 후 대기

# 예: 3번 옵션 선택 
dropdown3 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/span'))) 
dropdown3.click() 
# 뜨개질
option3 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[1]/a'))) 
option3.click()
time.sleep(1)  # 드롭다운 선택 후 대기

# 예: 4번 옵션 선택 
# 뜨개실 선택할 경우

# dropdown4 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[4]/span'))) 
# dropdown4.click() 
# 뜨개실
# option4 = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[4]/ul/li[1]/a'))) 
# option4.click()

# time.sleep(1)  # 드롭다운 선택 후 대기

# 기기별, 성별, 연령 전체 선택
driver.find_element(By.XPATH, '//*[@id="18_device_0"]').click()
driver.find_element(By.XPATH, '//*[@id="19_gender_0"]').click()

# 연령대 선택
age_checkbox = driver.find_element(By.XPATH, '//*[@id="20_age_0"]')
driver.execute_script("arguments[0].scrollIntoView(true);", age_checkbox)
age_checkbox.click()

time.sleep(2)  # 필요한 요소 로딩 대기

# '조회하기' 버튼 클릭
search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/a')))
search_button.click()

# 결과 스크래핑 및 Excel 파일 저장
for i in range(1):  # 페이지 수 조정
    # 페이지 로딩 대기
    time.sleep(1)
    
    for j in range(1, 21):  # 한 페이지당 최대 20개 아이템
        try:
            path = f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[{j}]/a'
            result = driver.find_element(By.XPATH, path).text
            print(result.split('\n'))
            ws.append(result.split('\n'))
        except Exception as e:
            print(f"Error occurred while scraping data: {e}")
    
    # 다음 페이지로 넘어가는 버튼 클릭 로직 (실제 페이지 구조에 따라 달라질 수 있음)
    try:
        next_button = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/a[2]')
        next_button.click()
        time.sleep(1)  # 페이지 로딩 대기
    except Exception as e:
        print(f"Error occurred while clicking next button: {e}")
        break

# 파일 저장
wb.save(file_path)
wb.close()

# 작업 완료 후, 드라이버 종료
driver.quit()

# 저장된 파일 열기
os.startfile(file_path)

print(f"'{excel_file_name}' 파일이 현재 폴더에 저장되었습니다.")

저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): 1
['1', '라라뜨개']
['2', '뜨개실']
['3', '수세미실']
['4', '니트컨테이너']
['5', '청송뜨개실']
['6', '뜨개질']
['7', '코바늘']
['8', '얀메이크']
['9', '손염색실']
['10', '콘사']
['11', '니트하임']
['12', '코바늘diy']
['13', '목도리뜨개질']
['14', '코와코이로이로']
['15', '뜨개사계절']
['16', '바늘이야기']
['17', '모헤어실']
['18', '털실']
['19', '코바늘가방']
['20', '야닝야닝']
'1_20240926.xlsx' 파일이 현재 폴더에 저장되었습니다.
